In [1]:
# SPARQL query to extract lead transitions
query = """
PREFIX crm: <http://www.example.org/crm_detailed_ontology#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?lead_id ?lead_name ?params_id ?old_status_id ?old_status_name ?new_status_id ?new_status_name ?change_date ?item_id
WHERE {
    ?params_class rdf:type crm:ParamsClass .
    ?lead crm:hasItemFromLead ?item .


    ?lead crm:hasID ?lead_id .
    ?lead crm:hasName ?lead_name .
    ?params_class crm:hasID ?params_id .

    ?params_class crm:belongFromParamsClassToOldStatus ?old_status .

    ?old_status crm:hasID ?old_status_id .
    ?old_status crm:hasName ?old_status_name .

    ?params_class crm:belongFromParamsClassToNewStatus ?new_status .
    ?new_status crm:hasID ?new_status_id .

    ?new_status crm:hasName ?new_status_name .

    ?item crm:hasID ?item_id .
    ?item crm:hasDateModified ?change_date .

}
ORDER BY ?lead_id ?change_date
"""

In [2]:
query_features = """
PREFIX crm: <http://www.example.org/crm_detailed_ontology#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?lead_id ?lead_name ?created_at ?responsible_user_id (COUNT(?item) AS ?interaction_count)
WHERE {
    ?lead rdf:type crm:Lead .
    ?lead crm:hasID ?lead_id .
    ?lead crm:hasName ?lead_name .
    ?lead crm:hasCreatedAt ?created_at .
    OPTIONAL { ?lead crm:hasResponsibleUserID ?responsible_user_id . }
    OPTIONAL { ?lead crm:hasItemFromLead ?item . }
}
GROUP BY ?lead_id ?lead_name ?created_at ?responsible_user_id
"""

In [3]:
# Import necessary libraries
from collections import defaultdict
from datetime import datetime, timedelta
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.plugins.sparql import prepareQuery

# Load the RDF graph
g = Graph()
g.parse('crm_graph.rdf', format='xml')

# Define the namespace
CRM = Namespace("http://www.example.org/crm_detailed_ontology#")
g.bind("crm", CRM)

In [4]:
# Execute the status transitions query
results = g.query(query)

# Process the results
transitions_data = []
for row in results:
    lead_id = row.lead_id.toPython()
    lead_name = row.lead_name.toPython()
    params_id = row.params_id.toPython()
    old_status_id = row.old_status_id.toPython()
    old_status_name = row.old_status_name.toPython()
    new_status_id = row.new_status_id.toPython()
    new_status_name = row.new_status_name.toPython()
    change_date = row.change_date.toPython()
    item_id = row.item_id.toPython()
    
    
    transitions_data.append({
        'lead_id': lead_id,
        'lead_name': lead_name,
        'params_id': params_id,
        'old_status_id': old_status_id,
        'old_status_name': old_status_name,
        'new_status_id': new_status_id,
        'new_status_name': new_status_name,
        'change_date': change_date,
        'item_id': item_id
    })

# Similarly execute and process the features query
results_features = g.query(query_features)

features_data = []
for row in results_features:
    lead_id = row.lead_id.toPython()
    lead_name = row.lead_name.toPython()
    created_at = row.created_at.toPython()
    responsible_user_id = row.responsible_user_id.toPython() if row.responsible_user_id else None
    interaction_count = int(row.interaction_count)
    
    features_data.append({
        'lead_id': lead_id,
        'lead_name': lead_name,
        'created_at': created_at,
        'responsible_user_id': responsible_user_id,
        'interaction_count': interaction_count
    })

In [5]:
import pandas as pd

# Convert lists to DataFrames
df_transitions = pd.DataFrame(transitions_data)
df_features = pd.DataFrame(features_data)

# Merge data on 'lead_id'
df_merged = pd.merge(df_transitions, df_features, on=['lead_id', 'lead_name'], how='left')

# Ensure 'change_date' is in datetime format
df_merged['change_date'] = pd.to_datetime(df_merged['change_date'])
df_merged['created_at'] = pd.to_datetime(df_merged['created_at'])

In [6]:
df_transitions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60388 entries, 0 to 60387
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   lead_id          60388 non-null  int64         
 1   lead_name        60388 non-null  object        
 2   params_id        60388 non-null  int64         
 3   old_status_id    60388 non-null  int64         
 4   old_status_name  60388 non-null  object        
 5   new_status_id    60388 non-null  int64         
 6   new_status_name  60388 non-null  object        
 7   change_date      60388 non-null  datetime64[ns]
 8   item_id          60388 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 4.1+ MB


In [7]:
df_merged.head()

,lead_id,lead_name,params_id,old_status_id,old_status_name,new_status_id,new_status_name,change_date,item_id,created_at,responsible_user_id,interaction_count
0,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,47620073,1970-01-01 00:00:01.602076820,11089494,200
1,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,47620074,1970-01-01 00:00:01.602076820,11089494,200
2,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,47620075,1970-01-01 00:00:01.602076820,11089494,200
3,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,47620076,1970-01-01 00:00:01.602076820,11089494,200
4,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,47620077,1970-01-01 00:00:01.602076820,11089494,200


In [8]:
df_merged.describe()

,lead_id,params_id,old_status_id,new_status_id,change_date,item_id,created_at,responsible_user_id,interaction_count
count,6.038800e+04,60388.000000,6.038800e+04,6.038800e+04,60388,6.038800e+04,60388,6.038800e+04,60388.000000
mean,2.698685e+07,264.870968,2.932182e+07,3.160860e+07,2023-10-21 21:50:59.080081920,4.636560e+07,1970-01-01 00:00:01.665628697,8.692636e+06,89.817248
min,2.405054e+07,12.000000,1.420000e+02,1.420000e+02,2020-10-07 16:20:20,1.000000e+00,1970-01-01 00:00:01.602076820,7.737097e+06,14.000000
25%,2.569082e+07,149.000000,3.463941e+07,1.430000e+02,2023-05-18 15:19:41,4.761819e+07,1970-01-01 00:00:01.629795367,7.737097e+06,45.000000
50%,2.773609e+07,307.000000,3.463941e+07,3.464989e+07,2024-02-20 05:10:50.500000,4.761981e+07,1970-01-01 00:00:01.679552085,7.737097e+06,82.000000
75%,2.792520e+07,376.000000,3.464903e+07,5.289391e+07,2024-08-22 12:20:54,4.762005e+07,1970-01-01 00:00:01.685633521,1.108949e+07,93.000000
max,2.878820e+07,490.000000,5.289391e+07,5.289391e+07,2024-12-19 16:17:26,4.762033e+07,1970-01-01 00:00:01.714049212,1.108949e+07,200.000000
std,1.667223e+06,137.962507,1.672471e+07,2.138987e+07,NaN,6.924573e+06,NaN,1.446659e+06,60.319156


In [9]:
# Define the time window in days
time_window = pd.Timedelta(days=7)

# Sort transitions by lead_id and change_date
df_merged.sort_values(by=['lead_id', 'change_date'], inplace=True)

# Initialize labels
df_merged['label'] = 0

# Group by lead
for lead_id, group in df_merged.groupby('lead_id'):
    transitions = group.reset_index()
    df_sorted_transitions = transitions.sort_values(by=['change_date'], ascending=False)
    # print(len(df_sorted_transitions['item_id'].sort_values().unique()))
    # df_sorted_transitions.to_csv('out.csv', sep='\t')
    # break
    counter = 0
    for i in range(len(transitions)):
        current_transition = transitions.loc[i]
        current_date = current_transition['change_date']
        current_new_status = current_transition['new_status_id']
        if i + 1 < len(transitions):
            next_transition = transitions.loc[i + 1]
            next_date = next_transition['change_date']
            next_new_status = next_transition['new_status_id']
            
            if next_new_status != current_new_status:
                counter+=1
                
                time_diff = next_date - current_date
                if time_diff <= time_window:
                    # Lead moved to next status within time window
                    df_merged.loc[current_transition['index'], 'label'] = 1
                else:
                    df_merged.loc[current_transition['index'], 'label'] = 0
        else:
            # No subsequent transition; label depends on business logic
            df_merged.loc[current_transition['index'], 'label'] = 0
    print("current_new_status", counter)
    

current_new_status 5129
current_new_status 1709
current_new_status 1319
current_new_status 2009
current_new_status 659
current_new_status 599
current_new_status 839
current_new_status 209
current_new_status 809
current_new_status 449
current_new_status 1139
current_new_status 269
current_new_status 1829
current_new_status 929
current_new_status 839
current_new_status 1019
current_new_status 2129


In [10]:
# Time since lead creation
df_merged['time_since_creation'] = (df_merged['change_date'] - df_merged['created_at']).dt.total_seconds() / (3600*24)

# Convert categorical variables
df_merged['responsible_user_id'] = df_merged['responsible_user_id'].astype('category')
df_merged['new_status_id'] = df_merged['new_status_id'].astype('category')

# For time since last status change, we can calculate the difference between current and previous change dates
df_merged['time_since_last_change'] = df_merged.groupby('lead_id')['change_date'].diff().dt.total_seconds() / (3600*24)
# Fill NaN with time since creation for the first transition
df_merged['time_since_last_change'].fillna(df_merged['time_since_creation'], inplace=True)

C:\Users\kirsr\AppData\Local\Temp\ipykernel_2872\2805160972.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['time_since_last_change'].fillna(df_merged['time_since_creation'], inplace=True)


In [11]:
# Convert categorical features to numerical codes
df_merged['responsible_user_code'] = df_merged['responsible_user_id'].cat.codes
df_merged['new_status_code'] = df_merged['new_status_id'].cat.codes

# Select features and label
feature_columns = [
    'time_since_creation',
    'time_since_last_change',
    'interaction_count',
    'responsible_user_code',
    'new_status_code',
    # Add other features if necessary
]
X = df_merged[feature_columns]
y = df_merged['label']

In [15]:
# Step 1: Determine training and test leads
# For example, split the leads into training and test sets
from sklearn.model_selection import train_test_split
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

import numpy as np

lead_ids = df_merged['lead_id'].unique()
train_lead_ids, test_lead_ids = train_test_split(lead_ids, test_size=0.2, random_state=42)

# Step 2: Filter triples to include only those related to training leads
training_triples = []
testing_triples = []

# Define maximum acceptable length for triple components
MAX_COMPONENT_LENGTH = 500  # Adjust as needed

# Reconstruct training_triples without excessively long components
filtered_training_triples = []
long_component_count = 0

for s, p, o in training_triples:
    if len(s) > MAX_COMPONENT_LENGTH or len(p) > MAX_COMPONENT_LENGTH or len(o) > MAX_COMPONENT_LENGTH:
        long_component_count += 1
        continue  # Skip this triple
    filtered_training_triples.append((s, p, o))

print(f"Excluded {long_component_count} triples with excessively long components.")
print(f"Remaining training triples: {len(filtered_training_triples)}")

# Step 3: Create TriplesFactories
training_tf = TriplesFactory.from_labeled_triples(
    triples=np.array(training_triples, dtype=str)
)

testing_tf = TriplesFactory.from_labeled_triples(
    triples=np.array(testing_triples, dtype=str)
)

# Step 4: Train embeddings using only the training triples
result = pipeline(
    training=training_tf,
    testing=testing_tf,  # Optionally include testing_tf for evaluation
    model='TransE',
    training_kwargs={
        'num_epochs': 100,
        'batch_size': 256
    },
    optimizer='adam',
    optimizer_kwargs={
        'lr': 0.001
    },
    random_seed=42,
    device='cpu'
)

MemoryError: Unable to allocate 139. GiB for an array with shape (37859, 3) and data type <U327687

In [103]:
# Извлечение эмбеддингов сущностей
entity_embedding_model = result.model.entity_representations[0]
entity_embeddings = entity_embedding_model().detach().cpu().numpy()

# Отображение сущностей на их индексы
entity_to_id = training_tf.entity_to_id
id_to_entity = {idx: entity for entity, idx in entity_to_id.items()}

# Создание словаря эмбеддингов
entity_to_embedding = {
    id_to_entity[idx]: embedding
    for idx, embedding in enumerate(entity_embeddings)
    if idx in id_to_entity
}

In [104]:
print(f"Embeddings shape: {entity_embeddings.shape}")

Embeddings shape: (2608, 50)


In [105]:
sample_entity = list(entity_to_embedding.keys())[0]
print(f"Entity: {sample_entity}")
print(f"Embedding: {entity_to_embedding[sample_entity]}")

Entity: http://www.example.org/crm_detailed_ontology
Embedding: [-0.21842453  0.1891045   0.07104044  0.0639934   0.13652307 -0.23349927
  0.16455984 -0.1806914  -0.20906341  0.1638591  -0.05124538 -0.12180431
 -0.10925232 -0.20701511 -0.00787001  0.1902629  -0.20272277  0.00648969
  0.09586602  0.19566487 -0.22180785  0.06628236 -0.04994451  0.01938166
  0.06443479  0.15064698 -0.03469307 -0.20918168 -0.14112233  0.16613883
 -0.12423296 -0.07692382  0.22069363  0.01008057  0.14966358  0.07731868
 -0.16526152 -0.01751232  0.15148734  0.13094124 -0.1750609  -0.10124443
 -0.04141213  0.13165377  0.14532083  0.10066251  0.12364255  0.12736236
 -0.1902651  -0.07663175]


In [106]:
test_list = []
for k, v in entity_to_embedding.items():
    test = 'http://www.example.org/crm_detailed_ontology#Status'
    if test in k:
        test_list.append(k)
print(test_list)

['http://www.example.org/crm_detailed_ontology#Status', 'http://www.example.org/crm_detailed_ontology#Status_10166691', 'http://www.example.org/crm_detailed_ontology#Status_10166697', 'http://www.example.org/crm_detailed_ontology#Status_10166700', 'http://www.example.org/crm_detailed_ontology#Status_10166727', 'http://www.example.org/crm_detailed_ontology#Status_10166730', 'http://www.example.org/crm_detailed_ontology#Status_142', 'http://www.example.org/crm_detailed_ontology#Status_143', 'http://www.example.org/crm_detailed_ontology#Status_17467777', 'http://www.example.org/crm_detailed_ontology#Status_17467780', 'http://www.example.org/crm_detailed_ontology#Status_17467783', 'http://www.example.org/crm_detailed_ontology#Status_17467927', 'http://www.example.org/crm_detailed_ontology#Status_17467930', 'http://www.example.org/crm_detailed_ontology#Status_17467933', 'http://www.example.org/crm_detailed_ontology#Status_17467936', 'http://www.example.org/crm_detailed_ontology#Status_17467

In [107]:
embedding_dimension = entity_embeddings.shape[1]

# Обработка идентификаторов лидов
def ensure_scalar(x):
    if isinstance(x, (list, np.ndarray)):
        return x[0] if len(x) > 0 else None
    else:
        return x

df_merged['lead_id'] = df_merged['lead_id'].apply(ensure_scalar)
df_merged['lead_uri'] = df_merged['lead_id'].apply(
    lambda x: str(CRM) + 'Lead_' + str(x)
)

# Получение эмбеддингов лидов
def get_entity_embedding(entity_uri):
    embedding = entity_to_embedding.get(entity_uri)
    if embedding is not None:
        return embedding
    else:
        print(embedding_dimension)
        return np.zeros(embedding_dimension)

df_merged['lead_embedding'] = df_merged['lead_uri'].apply(get_entity_embedding)

# Обработка идентификаторов статусов
df_merged['new_status_id'] = df_merged['new_status_id'].apply(ensure_scalar)
df_merged['status_uri'] = df_merged['new_status_id'].apply(
    lambda x: str(CRM) + 'Status_' + str(x)
)

# Получение эмбеддингов статусов
status_embedding = []
for i in df_merged['status_uri']:
    status_embedding.append(get_entity_embedding(i))
df_merged['status_embedding'] = status_embedding


In [108]:
# Кодирование статусов
df_merged['new_status_id'] = df_merged['new_status_id'].astype('category')
df_merged['new_status_code'] = df_merged['new_status_id'].cat.codes

In [109]:
df_merged.head()

,lead_id,lead_name,params_id,old_status_id,old_status_name,new_status_id,new_status_name,change_date,created_at,responsible_user_id,interaction_count,label,time_since_creation,time_since_last_change,responsible_user_code,new_status_code,lead_uri,lead_embedding,status_uri,status_embedding
0,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,1970-01-01 00:00:01.602076820,11089494,200,1,18542.680768,18542.680768,3,4,http://www.example.org/crm_detailed_ontology#L...,"[0.11592786, -0.1382676, 0.13578388, 0.0015336...",http://www.example.org/crm_detailed_ontology#S...,"[-0.030623786, -0.031060219, 0.12888238, -0.12..."
1,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,1970-01-01 00:00:01.602076820,11089494,200,1,18542.680768,0.000000,3,4,http://www.example.org/crm_detailed_ontology#L...,"[0.11592786, -0.1382676, 0.13578388, 0.0015336...",http://www.example.org/crm_detailed_ontology#S...,"[-0.030623786, -0.031060219, 0.12888238, -0.12..."
2,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,1970-01-01 00:00:01.602076820,11089494,200,1,18542.680768,0.000000,3,4,http://www.example.org/crm_detailed_ontology#L...,"[0.11592786, -0.1382676, 0.13578388, 0.0015336...",http://www.example.org/crm_detailed_ontology#S...,"[-0.030623786, -0.031060219, 0.12888238, -0.12..."
3,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,1970-01-01 00:00:01.602076820,11089494,200,1,18542.680768,0.000000,3,4,http://www.example.org/crm_detailed_ontology#L...,"[0.11592786, -0.1382676, 0.13578388, 0.0015336...",http://www.example.org/crm_detailed_ontology#S...,"[-0.030623786, -0.031060219, 0.12888238, -0.12..."
4,24050537,Proskater.Ru,278,41159701,возвращение в работу,34649023,Клиент квалифицирован,2020-10-07 16:20:20,1970-01-01 00:00:01.602076820,11089494,200,1,18542.680768,0.000000,3,4,http://www.example.org/crm_detailed_ontology#L...,"[0.11592786, -0.1382676, 0.13578388, 0.0015336...",http://www.example.org/crm_detailed_ontology#S...,"[-0.030623786, -0.031060219, 0.12888238, -0.12..."


In [110]:
# Проверка наличия всех необходимых признаков
feature_columns = [
    'time_since_creation',
    'time_since_last_change',
    'interaction_count',
    'responsible_user_code',
    'new_status_code'
]

missing_features = [col for col in feature_columns if col not in df_merged.columns]
if missing_features:
    print(f"Отсутствуют следующие признаки: {missing_features}")
else:
    print("Все необходимые признаки присутствуют.")


Все необходимые признаки присутствуют.


In [111]:
# Объединение эмбеддингов и других признаков
def combine_features(row):
    features = []
    # Эмбеддинги
    features.extend(row['lead_embedding'])
    features.extend(row['status_embedding'])
    # Другие признаки
    numeric_features = row[feature_columns].values.astype(float).tolist()
    features.extend(numeric_features)
    return features

df_merged['combined_features'] = df_merged.apply(combine_features, axis=1)

# Подготовка данных для модели
X = np.vstack(df_merged['combined_features'].values)
y = df_merged['label'].values.astype(int)


In [56]:
X

NameError: name 'X' is not defined

In [112]:
y

array([1, 1, 1, ..., 1, 1, 0], shape=(60388,))